# Utilizing SageMaker Notebook Job Steps To Operationalize ML Workflows

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

---

In this example we take a look at how we can utilize SageMaker Notebook Job Steps to build a simple end to end ML workflow including Feature Engineering, Training, and Evaluation. We will be utilizing an NLP BERT Model with the SST2 dataset for Text Classification. The SST2 dataset contains two columns including text and a classification column with 2 labels: 0 and 1.

This notebook creates and executes a pipeline with two notebook steps and three notebooks in its workflow:

- <b>Preprocess Step</b>: Pulls down the NLP Dataset and conducts preprocessing.
- <b>Train and Batch Inference Step</b>: Conducts Training and Batch Inference with a Transformers BERT Model.
    - Additional Dependencies: Notebook that prepares a test dataset for Batch Inference.

## Setup

For setup we install the SageMaker Python SDK and also need to ensure our role has the proper IAM permissions for the [Studio user](https://docs.aws.amazon.com/sagemaker/latest/dg/scheduled-notebook-policies-studio.html):

1. Create and execute the SageMaker Pipeline.
2. Notebook Job Execution Role passed to the Notebook Job Step that will execute the underlying Training Job.

In [ ]:
%pip install "sagemaker>=2.199.0,<3"

## Pipeline Setup

In [ ]:
import sagemaker
from sagemaker.workflow.notebook_job_step import NotebookJobStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker import session

sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
s3_bucket_prefix = "notebook-step-artifacts-pipelines/"
default_bucket_prefix = sagemaker_session.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    subfolder_name = f"{default_bucket_prefix}/{s3_bucket_prefix}"
else:
    subfolder_name = s3_bucket_prefix

image_uri = "542918446943.dkr.ecr.us-west-2.amazonaws.com/sagemaker-distribution-prod:0-cpu"
kernel_name = "python3"
role = sagemaker.get_execution_role()
notebook_artifacts = f"s3://{default_bucket}/{subfolder_name}"
notebook_artifacts

In [ ]:
pipeline_name = "nb-job-step-pipelines-demo"
display_name = "MyNotebookSteps"
preprocess_notebook = "preprocess.ipynb"
preprocess_job_name = "nb-preprocess"
preprocess_description = "This step downloads an NLP dataset and creates a CSV file out of it"
preprocess_step_name = "preprocess-bert"

# notebook job parameters
nb_job_params = {"default_s3_bucket": notebook_artifacts}

preprocess_nb_step = NotebookJobStep(
    name=preprocess_step_name,
    description=preprocess_description,
    notebook_job_name=preprocess_job_name,
    image_uri=image_uri,
    kernel_name=kernel_name,
    display_name=display_name,
    role=role,
    input_notebook=preprocess_notebook,
    instance_type="ml.m5.4xlarge",
    parameters=nb_job_params,
)
# notebook two configuration
training_notebook = "training.ipynb"
test_data_prep_notebook = "prepare-test-set.ipynb"
training_job_name = "nb-training"
training_description = "This step takes the prepared S3 dataset and runs fine-tuning"
training_step_name = "training-bert"

train_nb_step = NotebookJobStep(
    name=training_step_name,
    description=training_description,
    notebook_job_name=training_job_name,
    input_notebook=training_notebook,
    additional_dependencies=[test_data_prep_notebook],
    image_uri=image_uri,
    kernel_name=kernel_name,
    display_name=display_name,
    instance_type="ml.m5.12xlarge",
    role=role,
    parameters=nb_job_params,
)
train_nb_step.add_depends_on([preprocess_nb_step])

In [ ]:
# create pipeline
pipeline = Pipeline(
    name=pipeline_name,
    steps=[preprocess_nb_step, train_nb_step],
)

### Pipeline Execution

This pipeline will take approximately 20 minutes to execute.

In [ ]:
pipeline.upsert(session.get_execution_role())
execution = pipeline.start(parameters={})
execution.wait(delay=30, max_attempts=60)
execution_steps = execution.list_steps()
print(execution_steps)

## Output Notebook Parsing

In [ ]:
# download the output notebook

from sagemaker.s3_utils import s3_path_join
from sagemaker.utils import _tmpdir
from sagemaker.s3 import S3Downloader
import tarfile
import os


# get job details
def _get_training_job_details(notebook_job_step):
    training_job_arn = notebook_job_step["Metadata"]["TrainingJob"]["Arn"]

    return sagemaker_session.sagemaker_client.describe_training_job(
        TrainingJobName=training_job_arn.split("/")[1]
    )


def _download_notebook(output_s3_uri, output_notebook_name, kms_key=None):
    download_folder = "outputs"

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    with _tmpdir() as temp_output_folder:
        S3Downloader.download(
            output_s3_uri,
            temp_output_folder,
            sagemaker_session=sagemaker_session,
            kms_key=kms_key,
        )

        with tarfile.open(os.path.join(temp_output_folder, "output.tar.gz"), "r:gz") as tar:
            tar.extract(output_notebook_name, download_folder)
            print(f"Downloaded to {download_folder}/{output_notebook_name}")


# download the output notebook job
job_description = _get_training_job_details(execution_steps[0])

output_s3_uri = s3_path_join(
    job_description["OutputDataConfig"]["S3OutputPath"],
    job_description["TrainingJobName"],
    "output",
    "output.tar.gz",
)
output_notebook_name = job_description["Environment"]["SM_OUTPUT_NOTEBOOK_NAME"]

print(f"  - Output S3 Location: {output_s3_uri}")
print(f"  - Output Notebook Name: {output_notebook_name}")

_download_notebook(output_s3_uri, output_notebook_name)

## Cleanup

In [ ]:
# Delete the Pipeline
pipeline.delete()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-pipelines|notebook-job-step|nb-job-pipeline.ipynb)
